In [ ]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, SSN, SOSA #most common namespaces
import urllib.parse #for parsing strings to URI's
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
# import stardog

In [ ]:
df = pd.read_csv('Jasmin.csv')

In [ ]:
df['Altitude']= pd.to_numeric(df['Altitude'],errors='coerce')
df['Altitude'].mean()

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
g = Graph()
ID = Namespace('foo_')
owl = Namespace('http://www.w3.org/2002/07/owl#')
sosa = Namespace('http://www.w3.org/ns/sosa/')
foo = Namespace('http://www.ontology/ns/foo/1.1#')
lat = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
long =Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
alt = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
time = Namespace('http://www.w3.org/2006/time#')
schema = Namespace('http://schema.org/')
XSD=Namespace('http://www.w3.org/2001/XMLSchema#')
Class = Namespace('http://purl.org/ontology/wo/Class/')
Family = Namespace('http://purl.org/ontology/wo/Family/')
Kingdom = Namespace('http://purl.org/ontology/wo/kingdom/')
Order = Namespace('http://purl.org/ontology/wo/Proboscidea/')
Phylum = Namespace('http://purl.org/ontology/wo/Phylum/')

In [ ]:
for index, row in df.iterrows():
    g.add((URIRef(foo+row['ID']), RDF.type, SOSA.Observation))
    g.add((URIRef(ID+row['ID']), foo.ID, Literal(row['ID'], datatype=XSD.string))) 

    g.add((URIRef(schema+'DGFC/elephant#Jasmin'), RDF.type,  sosa.Sensor))
    g.add((URIRef(ID+row['ID']), URIRef(schema+'dgfc/elephant#Jasmin'), Literal(row['ID'], datatype=XSD.string))) 
          
    g.add((URIRef(foo+'LocalDate'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.LocalDate, Literal(row['LocalDate'], datatype=XSD.date)))
    
    g.add((URIRef(foo+'LocalTime'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']),  time.LocalTime, Literal(row['LocalTime'], datatype=XSD.time)))
      
    g.add((URIRef(foo+'GMTDate'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.GMTDate, Literal(row['GMTDate'], datatype=XSD.date)))

    g.add((URIRef(foo+'GMTTime'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), time.GMTTime,  Literal(row['GMTTime'], datatype=XSD.time)))

    g.add((URIRef(lat+'lat'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), lat.lat, Literal(row['lat'], datatype=XSD.float)))
     
    g.add((URIRef(long+'long'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), long.long, Literal(row['long'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'Temperature'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Temperature, Literal(row['Temperature'], datatype=XSD.double)))
    
    g.add((URIRef(foo+'Speed'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Speed, Literal(row['Speed'], datatype=XSD.float)))
    
    g.add((URIRef(alt+'alt'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']),  alt.alt, Literal(row['Altitude'], datatype=XSD.float)))
   
    g.add((URIRef(foo+'Direction'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Direction, Literal(row['Direction'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'Distance'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Distance,  Literal(row['Distance'], datatype=XSD.float)))
    
    g.add((URIRef(foo+'HDOP'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.HDOP, Literal(row['HDOP'], datatype=XSD.integer)))
 
    g.add((URIRef(foo+'Cov'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Cov, Literal(row['Cov'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+'Count'), RDF.type, sosa.observedProperty))
    g.add((URIRef(ID+row['ID']), foo.Count, Literal(row['Count'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Class.Mammalia))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string)))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Family.Elephantidae))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string))) 
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Kingdom.Animalia))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string))) 
     
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Phylum.Chordata))
    g.add((URIRef(ID+row['ID']), URIRef(foo+'ElephasMaximus'), Literal(row['ID'], datatype=XSD.string)))
    

In [ ]:
print(g.serialize(format='turtle')).head(1)

In [ ]:
# saving ontology to disk
g.serialize("Jasmin10march.rdf", format="ttl")